# Calculo de la Elasticidad y Precio Optimo

In [1]:
import pandas as pd

trans = pd.read_csv("eci_transactions.csv")

In [2]:
trans.sort_values(["SKU", "DATE"], ascending=[True, True])

,TRANSACTION_ID,DATE,STORE_ID,SKU,QUANTITY,PRICE,TOTAL_SALES,SUBGROUP,STORE_SUBGROUP_DATE_ID
2298,10066,2021-01-01,S00086,BEAHASH001,1.0,33.52,33.52,Shampoo,S00086_Shampoo_2021-01-01
3478,7907,2021-01-01,S00068,BEAHASH001,4.0,35.53,142.12,Shampoo,S00068_Shampoo_2021-01-01
4967,14500,2021-01-01,S00124,BEAHASH001,1.0,37.61,37.61,Shampoo,S00124_Shampoo_2021-01-01
6082,7909,2021-01-01,S00068,BEAHASH001,1.0,35.53,35.53,Shampoo,S00068_Shampoo_2021-01-01
7793,16301,2021-01-01,S00140,BEAHASH001,1.0,34.51,34.51,Shampoo,S00140_Shampoo_2021-01-01
...,...,...,...,...,...,...,...,...,...
19001178,19002851,2023-12-31,S00138,TOYGAPU006,1.0,24.74,24.74,Puzzles,S00138_Puzzles_2023-12-31
19001493,18995788,2023-12-31,S00067,TOYGAPU006,1.0,24.15,24.15,Puzzles,S00067_Puzzles_2023-12-31
19002342,19000034,2023-12-31,S00110,TOYGAPU006,1.0,25.01,25.01,Puzzles,S00110_Puzzles_2023-12-31
19002843,18994576,2023-12-31,S00055,TOYGAPU006,1.0,25.34,25.34,Puzzles,S00055_Puzzles_2023-12-31


In [3]:
trans["SKU"].nunique()

854

In [5]:
import numpy as np
import statsmodels.api as sm

def elasticidad_historica(df, sku,  metodo="regresion"):
    # Filtrar datos del SKU
    data = df[df["SKU"] == sku].copy()
    
    # Agrupar por fecha (si hay varias observaciones en un día)
    daily = data.groupby("DATE").agg({
        "QUANTITY": "sum",
        "PRICE": "mean"
    }).reset_index()
    
    if metodo == "arc":
        Q_max, Q_min = daily["QUANTITY"].max(), daily["QUANTITY"].min()
        P_max, P_min = daily["PRICE"].max(), daily["PRICE"].min()
        Q_avg = (Q_max + Q_min) / 2
        P_avg = (P_max + P_min) / 2
        
        elasticidad = ((Q_max - Q_min)/Q_avg) / ((P_max - P_min)/P_avg)
        return elasticidad
    
    elif metodo == "regresion":
        # Regresión log-log
        daily = daily[(daily["QUANTITY"] > 0) & (daily["PRICE"] > 0)]
        logQ = np.log(daily["QUANTITY"])
        logP = np.log(daily["PRICE"])
        
        logP = sm.add_constant(logP)  # intercepto
        model = sm.OLS(logQ, logP).fit()
        
        return model.params[1]  # coeficiente asociado a log(P)

In [6]:
# Elasticidad histórica por método del arco
e_arc = elasticidad_historica(trans, "BEAHASH001", metodo="arc")

# Elasticidad histórica con regresión log-log
e_reg = elasticidad_historica(trans, "BEAHASH001", metodo="regresion")

print("Elasticidad (Arc):", e_arc)
print("Elasticidad (Regresión):", e_reg)

Elasticidad (Arc): 2.8328154936032153
Elasticidad (Regresión): -1.701621819721874


In [7]:
def calcular_elasticidades(df):
    resultados = []
    
    # Iteramos por cada combinación única de SKU y STORE_ID
    for sku in df["SKU"].unique():
        
        # Elasticidad por arco
        e_arc = elasticidad_historica(df, sku, metodo="arc")
        
        # Elasticidad por regresión
        e_reg = elasticidad_historica(df, sku, metodo="regresion")
        
        # Guardamos en lista
        resultados.append({
            "SKU": sku,
            "Elasticidad_arc": e_arc,
            "Elasticidad_regresion": e_reg
        })
    
    # Convertimos a DataFrame
    return pd.DataFrame(resultados)

In [8]:
elasticidad_df = calcular_elasticidades(trans)

elasticidad_df.sort_values("Elasticidad_regresion")

,SKU,Elasticidad_arc,Elasticidad_regresion
580,CLOCHBO006,7.820403,-3.559531
587,GROPABA014,8.841074,-3.496884
44,CLOWOTO002,9.464870,-3.400307
410,CLOWOOU009,4.985022,-3.274246
686,CLOWOBO008,5.264026,-3.252475
...,...,...,...
797,ELEAUSP005,3.319925,-0.872677
488,ELEAUSP010,2.964723,-0.813602
813,CLOMEJA002,3.550783,-0.710845
258,CLOWOBO012,3.656446,-0.616236


In [ ]:
# Encontrar la última fecha para cada SKU
ultima_fecha = trans.groupby("SKU")["DATE"].max().reset_index()
ultima_fecha = ultima_fecha.rename(columns={"DATE": "LAST_DATE"})

# Unir para traer la info completa y filtrar por última fecha
df_merged = trans.merge(ultima_fecha, on="SKU")
df_merged = df_merged[df_merged["DATE"] == df_merged["LAST_DATE"]]

In [11]:
# Calcular precio promedio y desvío estándar por SKU en la última fecha
precio_stats = df_merged.groupby(["SKU", "LAST_DATE"])["PRICE"].agg(
    AVG_LAST_PRICE="mean",
    STD_LAST_PRICE="std"
).reset_index()

In [12]:
precio_stats.sort_values("STD_LAST_PRICE", ascending=False)

,SKU,LAST_DATE,AVG_LAST_PRICE,STD_LAST_PRICE
439,ELEMOSM012,2023-12-31,348.801429,24.330881
366,ELEAUHE011,2023-12-31,278.702593,24.129605
421,ELEMOAC003,2023-12-31,340.895714,22.739850
449,ELEMOWE010,2023-12-31,357.687000,21.307773
451,ELEMOWE012,2023-12-31,323.925909,20.612589
...,...,...,...,...
506,GROPABA006,2023-12-31,6.465500,0.170556
468,GROBECO017,2023-12-31,8.890000,0.027386
709,HOMFULI008,2023-12-31,94.620000,0.000000
474,GROBECO023,2023-12-31,9.940000,NaN


In [13]:
# Antes de unir, nos quedamos sólo con las columnas necesarias del df de precios
precio_promedio_simple = precio_stats[["SKU", "AVG_LAST_PRICE", "STD_LAST_PRICE"]]

datos_precio_elasticidad = precio_promedio_simple.merge(
    elasticidad_df,
    on="SKU",
    how="inner"   # o "left" si querés mantener todos los SKU de precio_promedio
)

In [14]:
# Creamos el campo con el factor de ajuste
datos_precio_elasticidad["Factor_Ajuste"] = datos_precio_elasticidad["Elasticidad_regresion"] / (datos_precio_elasticidad["Elasticidad_regresion"] + 1)

# Calculamos el precio óptimo sugerido
datos_precio_elasticidad["Precio_Optimo"] = datos_precio_elasticidad["AVG_LAST_PRICE"] * datos_precio_elasticidad["Factor_Ajuste"]

In [18]:
datos_precio_elasticidad.sort_values("STD_LAST_PRICE", ascending=False).head(10)

,SKU,AVG_LAST_PRICE,STD_LAST_PRICE,Elasticidad_arc,Elasticidad_regresion,Factor_Ajuste,Precio_Optimo
439,ELEMOSM012,348.801429,24.330881,2.991486,-1.824027,2.213552,772.090054
366,ELEAUHE011,278.702593,24.129605,3.122870,-1.795335,2.257332,629.124207
421,ELEMOAC003,340.895714,22.739850,4.145142,-1.449838,3.223023,1098.714664
449,ELEMOWE010,357.687000,21.307773,2.948723,-1.341226,3.930611,1405.928379
451,ELEMOWE012,323.925909,20.612589,3.786713,-2.324304,1.755114,568.526851
436,ELEMOSM009,261.136667,20.555923,4.000932,-1.727433,2.374698,620.120714
386,ELEAUSP008,353.804412,20.189645,3.187180,-1.077877,13.840790,4896.932496
384,ELEAUSP006,271.385000,20.062922,3.611431,-1.546717,2.829102,767.775725
397,ELECODE008,305.710000,19.721287,4.954123,-1.615577,2.624493,802.333699
369,ELEAUHE014,266.553333,18.861608,3.006365,-1.289140,4.458536,1188.437513
